# ENGR 010 Group Project - Mechanical Engineering Option
## Vehicle Dynamics Analysis Project
## Overview
In this group project, you will develop a Python-based application to analyze vehicle performance data and simulate basic vehicle dynamics. Your team will create an interactive dashboard that mechanical engineers could use to evaluate vehicle performance metrics, analyze efficiency, and visualize motion characteristics.

One important part of this project is the use of Generative AI tools like ChatGPT to complete the assignment. AI tools are going to become increasingly useful and more widely used in the job market you're headed into, and we want you to be prepared for it.  So, while you need to understand everything the AI creates on your behalf, for this assignment we are not only allowing you to use AI, we are requiring that you make use of it.  Let's see what amazing work you can create when you use AI as your personal assistant.
## Project Description
### Background
Understanding vehicle dynamics is crucial for automotive design and performance optimization. Mechanical engineers regularly analyze data from vehicle tests to assess performance, efficiency, and safety characteristics. This project simulates a simplified version of the tools used in automotive engineering practice.

### Project Specifications
Your team will create a Python application that:

1. Data Management
   - Imports vehicle test data from CSV files
   - Stores measurements including:
     * Speed
     * Acceleration
     * Engine RPM
     * Fuel consumption
     * Distance traveled
   - Handles multiple test runs and driving conditions

2. Analysis Features
   - Calculate basic statistics (mean, median, standard deviation)
   - Determine acceleration characteristics
   - Calculate fuel efficiency metrics
   - Analyze braking performance
   - Perform basic motion analysis

3. Visualization Features
   - Time series plots of vehicle parameters
   - Speed-acceleration curves
   - Fuel efficiency maps
   - Interactive dashboard for performance monitoring

### Sample Data
Sample data will be provided from three vehicle test runs, with measurements taken at 0.1-second intervals during various driving conditions. 

## Learning Objectives & Requirements
### Learning Objectives
- Apply fundamental Python programming concepts in a real-world context
- Gain experience with scientific computing libraries
- Develop skills in data visualization and analysis
- Practice collaborative software development
- Experiment with using Generative AI to assist with writing code, developing concepts, etc.

### Technical Requirements
Your project must implement the following Python concepts:
1. Variables and Data Types
   - Appropriate use of numerical data types for measurements
   - Clear and consistent variable naming
   - Use of constants for physical parameters

2. Control Structures
   - Multiple if/elif/else decision structures
   - At least two different types of loops (for, while)
   - Error handling using try/except

3. Functions and Organization
   - Minimum of 5 custom functions
   - Proper use of parameters and return values
   - Documentation using docstrings
   - Code organization into multiple Python files

4. Data Structures
   - Lists for storing time-series data
   - Dictionaries for storing vehicle parameters
   - NumPy arrays for physics calculations

5. Scientific Computing
   - Motion and force calculations using NumPy
   - Data manipulation using Pandas
   - At least 3 different types of plots using Matplotlib
   - Basic dynamics calculations

### Submission Requirements
1. Jupyter Notebook File Submission
   - Code used to complete project
   - Written descriptions that complement code and explain thought process (similar to Engineering Assignments)
   - If you want to turn in your code in a different format - i.e. a bunch of python files - that is absolutely allowed but not required.

2. Progress Reports
   - Two progress reports (templates in Section 3)
   - Due at weeks 9 and 12 of the semester

3. AI Prompts with Annotations
   - You should also submit any and all prompts to ChatGPT or other Gen-AI systems (e.g. Claude, Gemini, etc.) that you used for this project
   - Please annotate each prompt with a short description that describes your thought process for that inquiry. Some potential things to write about in those annotations:
      - Why did you ask AI that question? 
      - Did AI answer the question you asked well? How do you know? (e.g. the answer matches what we learned in the class on loops)
      - If you updated a prompt after getting an unsatisfactory answer, what was your thought process for the updated prompt?

4. Reflection on using AI for programming
   - Did you enjoy using AI to help with this project? 
   - What type of questions/prompts did AI work well for? What about things it didn't do well?
   - Did using AI save you time?
   - Were the concepts used in the AI-generated code things that you already had seen and understood? If not, why did you decide to use them in your final project? 

5. Final Presentation
   - 5-minute demonstration
   - Short Q&A
   - All team members must participate

### Grading Breakdown
- Functionality and Implementation (20%)
- AI prompts, annotations, and reflection (20%)
- Creativity & innovation (20%)
- Documentation (15%)
- Progress Reports (10%)
- Final Presentation (15%)

### Progress Reports
By the first progress report, teams should have:
- Basic data import functionality
- Initial power calculations
- Text output of some calculations

By the second progress report, teams should have:
- Complete analysis features
- Initial visualizations 
- Draft report

## Progress Report Template

### Progress Report Format

#### 1. Team Information
- Team Number:
- Team Members:
- Report Date:
- Report Number (1 or 2):

#### 2. Accomplishments
- List completed features and functionality
- Describe any challenges overcome
- Include screenshots or examples

#### 3. Current Challenges
- Describe ongoing technical issues
- List any resource needs
- Identify any team dynamics challenges

#### 4. Next Steps
- Detail planned work for next period
- Assign responsibilities
- Set specific goals and deadlines

#### 5. Timeline Update
- Compare progress against original timeline
- Explain any deviations
- Provide adjusted timeline if necessary

#### 6. Resource Utilization
- List Python packages/libraries used
- Document external resources consulted
- Note any additional tools needed

#### 7. Questions for Instructor
- List any technical questions
- Note any clarifications needed
- Request specific guidance if required

Progress reports should be submitted through the course management system. Each report should be 1-2 pages in length, and can include screenshots or code snippets, if relevant.

### Generates required data file

In [ ]:
import numpy as np
import pandas as pd
import os

def generate_speed_profile(duration, profile_type, max_speed=120):
    """
    Generate a speed profile based on the specified type.
    
    Parameters:
    duration (float): Duration of the test in seconds
    profile_type (str): Type of driving profile ('highway', 'urban', 'mixed')
    max_speed (float): Maximum speed in km/h
    
    Returns:
    numpy.ndarray: Array of speed values at 0.1s intervals
    """
    # Create time array with 0.1s intervals
    time_points = int(duration * 10)
    time = np.linspace(0, duration, time_points)
    
    if profile_type == 'highway':
        # Highway profile: acceleration to high speed, maintain, some variations
        # Acceleration phase
        accel_time = time[time <= 30]
        accel_speed = max_speed * (1 - np.exp(-accel_time/10))
        
        # Cruising phase with small variations
        cruise_time = time[time > 30]
        cruise_speed = max_speed + np.sin(cruise_time/5) * 5
        
        # Combine phases
        speed = np.concatenate([accel_speed, cruise_speed])
        
        # Add some random variation
        speed += np.random.normal(0, 2, len(speed))
        
    elif profile_type == 'urban':
        # Urban profile: multiple accelerations and decelerations
        speed = np.zeros_like(time)
        
        # Create several stop-and-go cycles
        for i in range(int(duration/60)):
            start_idx = int(i * 60 * 10)
            end_idx = int((i + 1) * 60 * 10)
            
            if end_idx > len(speed):
                end_idx = len(speed)
                
            cycle_time = time[start_idx:end_idx] - time[start_idx]
            
            # Each cycle: accelerate, cruise, decelerate, stop
            cycle_speed = 50 * np.sin(cycle_time * np.pi/60)**2
            speed[start_idx:end_idx] = cycle_speed
        
        # Add random variations
        speed += np.random.normal(0, 1, len(speed))
        
    else:  # mixed
        # Mixed profile: combination of highway and urban segments
        speed = np.zeros_like(time)
        
        # Divide into segments
        segment_duration = 120  # 2 minutes per segment
        num_segments = int(duration / segment_duration)
        
        for i in range(num_segments):
            start_idx = int(i * segment_duration * 10)
            end_idx = int((i + 1) * segment_duration * 10)
            
            if end_idx > len(speed):
                end_idx = len(speed)
                
            segment_time = end_idx - start_idx
            
            # Alternate between highway and urban patterns
            if i % 2 == 0:  # Highway-like segment
                segment_speed = 100 + 20 * np.sin(np.linspace(0, np.pi, segment_time))
                segment_speed += np.random.normal(0, 3, segment_time)
            else:  # Urban-like segment
                segment_speed = np.zeros(segment_time)
                for j in range(int(segment_time/600)):
                    stop_start = int(j * 600)
                    stop_end = int((j + 1) * 600)
                    
                    if stop_end > segment_time:
                        stop_end = segment_time
                        
                    cycle_length = stop_end - stop_start
                    cycle_time = np.linspace(0, 1, cycle_length)
                    segment_speed[stop_start:stop_end] = 60 * np.sin(cycle_time * np.pi)**2
                    
            speed[start_idx:end_idx] = segment_speed
    
    # Ensure no negative speeds
    speed = np.maximum(speed, 0)
    
    return speed

def calculate_acceleration(speed, time_step=0.1):
    """
    Calculate acceleration from speed data.
    
    Parameters:
    speed (numpy.ndarray): Array of speed values in km/h
    time_step (float): Time step in seconds
    
    Returns:
    numpy.ndarray: Array of acceleration values in m/s²
    """
    # Convert speed from km/h to m/s
    speed_ms = speed * (1000/3600)
    
    # Calculate acceleration (dv/dt)
    acceleration = np.zeros_like(speed_ms)
    acceleration[1:] = (speed_ms[1:] - speed_ms[:-1]) / time_step
    
    # Smooth acceleration to reduce noise
    window_size = 5
    smoothed_acceleration = np.convolve(acceleration, np.ones(window_size)/window_size, mode='same')
    
    return smoothed_acceleration

def calculate_engine_rpm(speed, gear_ratios, final_drive_ratio, wheel_diameter=0.7):
    """
    Calculate engine RPM based on vehicle speed and gear ratios.
    
    Parameters:
    speed (numpy.ndarray): Array of speed values in km/h
    gear_ratios (list): List of gear ratios
    final_drive_ratio (float): Final drive ratio
    wheel_diameter (float): Wheel diameter in meters
    
    Returns:
    numpy.ndarray: Array of engine RPM values
    """
    # Convert speed from km/h to m/s
    speed_ms = speed * (1000/3600)
    
    # Calculate wheel RPM: v = ω * r => ω = v / r
    wheel_rpm = speed_ms / (wheel_diameter/2) * 60 / (2 * np.pi)
    
    # Determine appropriate gear based on speed
    rpm = np.zeros_like(speed)
    
    # Simple gear selection based on speed thresholds
    speed_thresholds = [0, 20, 40, 60, 80, 110]
    
    for i in range(len(speed)):
        gear = 1  # Start with 1st gear
        
        # Select gear based on speed
        for j in range(len(speed_thresholds)):
            if speed[i] >= speed_thresholds[j] and j < len(gear_ratios):
                gear = j + 1
                
        # Calculate engine RPM using selected gear
        if gear <= len(gear_ratios):
            gear_ratio = gear_ratios[gear-1]
            rpm[i] = wheel_rpm[i] * gear_ratio * final_drive_ratio
    
    # Add some random variation to simulate real-world conditions
    rpm += np.random.normal(0, 50, len(rpm))
    
    # Ensure minimum RPM at idle
    min_rpm = 800
    rpm = np.maximum(rpm, min_rpm)
    
    return rpm

def calculate_fuel_consumption(speed, acceleration, rpm, engine_efficiency=0.35, vehicle_mass=1500):
    """
    Calculate instantaneous fuel consumption based on speed, acceleration and RPM.
    
    Parameters:
    speed (numpy.ndarray): Array of speed values in km/h
    acceleration (numpy.ndarray): Array of acceleration values in m/s²
    rpm (numpy.ndarray): Array of engine RPM values
    engine_efficiency (float): Engine thermal efficiency
    vehicle_mass (float): Vehicle mass in kg
    
    Returns:
    numpy.ndarray: Array of fuel consumption values in L/100km
    """
    # Convert speed from km/h to m/s
    speed_ms = speed * (1000/3600)
    
    # Calculate power required to overcome resistance and acceleration
    # P = F * v
    # F = m * a + F_rolling + F_air
    
    # Rolling resistance coefficient
    c_r = 0.015
    
    # Air resistance constants
    rho_air = 1.225  # kg/m³
    c_d = 0.3  # Drag coefficient
    A = 2.2  # Frontal area in m²
    
    # Calculate forces
    F_rolling = c_r * vehicle_mass * 9.81  # Rolling resistance
    F_air = 0.5 * rho_air * c_d * A * speed_ms**2  # Air resistance
    F_accel = vehicle_mass * acceleration  # Force required for acceleration
    
    # Total force
    F_total = F_rolling + F_air + F_accel
    
    # Power required (W)
    power = F_total * speed_ms
    
    # Convert negative power (braking) to zero fuel consumption
    power = np.maximum(power, 0)
    
    # Add idle fuel consumption when speed is very low
    idle_fuel_rate = 0.5  # L/h
    idle_condition = speed < 3
    
    # Energy in fuel (gasoline: ~34.8 MJ/L)
    fuel_energy_density = 34.8e6  # J/L
    
    # Calculate instantaneous fuel consumption in L/s
    fuel_rate = power / (fuel_energy_density * engine_efficiency)
    
    # Add idle consumption
    fuel_rate[idle_condition] = idle_fuel_rate / 3600  # Convert L/h to L/s
    
    # Convert to L/100km (standard fuel economy measure)
    # L/100km = (L/s) / (km/s) * 100
    # km/s = (km/h) / 3600
    
    # Avoid division by zero
    speed_kms = speed / 3600  # Convert to km/s
    speed_kms = np.maximum(speed_kms, 1e-6)  # Avoid division by zero
    
    fuel_consumption = fuel_rate / speed_kms * 100
    
    # Cap unrealistically high values that might occur at very low speeds
    fuel_consumption = np.minimum(fuel_consumption, 50)
    
    # Add some random variation
    fuel_consumption += np.random.normal(0, 0.5, len(fuel_consumption))
    fuel_consumption = np.maximum(fuel_consumption, 0)  # Ensure non-negative
    
    return fuel_consumption

def calculate_distance(speed, time_step=0.1):
    """
    Calculate cumulative distance based on speed data.
    
    Parameters:
    speed (numpy.ndarray): Array of speed values in km/h
    time_step (float): Time step in seconds
    
    Returns:
    numpy.ndarray: Array of cumulative distance values in meters
    """
    # Convert speed from km/h to m/s
    speed_ms = speed * (1000/3600)
    
    # Calculate distance increments: d = v * dt
    distance_increments = speed_ms * time_step
    
    # Calculate cumulative distance
    distance = np.cumsum(distance_increments)
    
    return distance

def generate_vehicle_test_data(duration, profile_type, vehicle_type='sedan'):
    """
    Generate complete vehicle test data for a specific profile and vehicle type.
    
    Parameters:
    duration (float): Duration of the test in seconds
    profile_type (str): Type of driving profile ('highway', 'urban', 'mixed')
    vehicle_type (str): Type of vehicle ('sedan', 'suv', 'sports')
    
    Returns:
    pandas.DataFrame: DataFrame containing all test data
    """
    # Number of data points
    time_points = int(duration * 10)
    
    # Create time array with 0.1s intervals
    time = np.linspace(0, duration, time_points)
    
    # Vehicle parameters based on type
    if vehicle_type == 'sedan':
        max_speed = 180  # km/h
        vehicle_mass = 1500  # kg
        engine_efficiency = 0.35
        gear_ratios = [3.5, 2.0, 1.5, 1.0, 0.75]
        final_drive_ratio = 3.7
        
    elif vehicle_type == 'suv':
        max_speed = 160  # km/h
        vehicle_mass = 2200  # kg
        engine_efficiency = 0.32
        gear_ratios = [3.8, 2.2, 1.6, 1.0, 0.7]
        final_drive_ratio = 4.1
        
    else:  # sports car
        max_speed = 250  # km/h
        vehicle_mass = 1300  # kg
        engine_efficiency = 0.38
        gear_ratios = [3.2, 2.0, 1.4, 1.0, 0.8, 0.6]
        final_drive_ratio = 3.5
    
    # Generate speed profile
    speed = generate_speed_profile(duration, profile_type, max_speed)
    
    # Calculate other parameters
    acceleration = calculate_acceleration(speed)
    engine_rpm = calculate_engine_rpm(speed, gear_ratios, final_drive_ratio)
    fuel_consumption = calculate_fuel_consumption(speed, acceleration, engine_rpm, engine_efficiency, vehicle_mass)
    distance = calculate_distance(speed)
    
    # Create DataFrame
    data = pd.DataFrame({
        'Time': time,
        'Speed': speed,  # km/h
        'Acceleration': acceleration,  # m/s²
        'EngineRPM': engine_rpm,  # RPM
        'FuelConsumption': fuel_consumption,  # L/100km
        'Distance': distance  # m
    })
    
    # Add test metadata
    data['TestID'] = f"{vehicle_type}_{profile_type}"
    data['VehicleType'] = vehicle_type
    data['ProfileType'] = profile_type
    
    return data

def main():
    """
    Generate sample data for three vehicle test runs and save to CSV.
    """
    # Create output directory if it doesn't exist
    if not os.path.exists('sample_data'):
        os.makedirs('sample_data')
    
    # Generate data for three different test runs
    test_scenarios = [
        {'duration': 600, 'profile': 'highway', 'vehicle': 'sedan'},
        {'duration': 900, 'profile': 'urban', 'vehicle': 'suv'},
        {'duration': 1200, 'profile': 'mixed', 'vehicle': 'sports'}
    ]
    
    # Combine all test data into one DataFrame
    all_data = pd.DataFrame()
    
    for scenario in test_scenarios:
        print(f"Generating {scenario['vehicle']} {scenario['profile']} data...")
        test_data = generate_vehicle_test_data(
            duration=scenario['duration'],
            profile_type=scenario['profile'],
            vehicle_type=scenario['vehicle']
        )
        all_data = pd.concat([all_data, test_data], ignore_index=True)
    
    # Save combined data to CSV
    filename = f"sample_data/vehicle_dynamics_data.csv"
    all_data.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

main()

### Our code

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Loading
BRAKING_THRESHOLD = -3.0
VEHICLE_SPECS = {
    'sedan': {'mass': 1500, 'efficiency': 0.35},
    'suv': {'mass': 2200, 'efficiency': 0.32},
    'sports': {'mass': 1300, 'efficiency': 0.38}
}

# Constants and Vehicle Specifications
def load_vehicle_data(filepath):
    try:
        return pd.read_csv(filepath)
    except FileNotFoundError:
        print(f"File {filepath} not found.")
        return None

# Statistical Summary
def compute_summary_stats(data, label):
    print(f"\n===== {label} =====")
    for column in ['Speed', 'Acceleration', 'EngineRPM', 'FuelConsumption', 'Distance']:
        stats = data[column].describe()
        print(f"{column}:")
        print(f"  Mean   : {stats['mean']:.2f}")
        print(f"  Median : {stats['50%']:.2f}")
        print(f"  Std Dev: {stats['std']:.2f}")

# Braking Analysis
def braking_events(data, threshold=BRAKING_THRESHOLD):
    hard_brakes = data[data['Acceleration'] < threshold]
    print(f"{len(hard_brakes)} braking events detected (acceleration < {threshold} m/s²)")
    return hard_brakes

# Fuel Efficiency Calculation
def calculate_fuel_efficiency(data):
    total_distance_km = data['Distance'].sum() / 1000
    total_fuel_liters = data['FuelConsumption'].sum()
    if total_fuel_liters == 0:
        return 0
    return total_distance_km / total_fuel_liters

# Force Calculation (Dynamics)
def calculate_force(data, vehicle_type):
    mass = VEHICLE_SPECS[vehicle_type]['mass']
    data['Force'] = mass * data['Acceleration']
    return data

# Motion Analysis
def estimate_distance_from_speed(data):
    distance = np.trapz(data['Speed'], data['Time']) / 3.6
    return distance

# Visualization Functions
def plot_vehicle_data(test_data, title):
    fig, axs = plt.subplots(4, 1, figsize=(12, 14), sharex=True)
    params = ['Speed', 'Acceleration', 'EngineRPM', 'Distance']
    colors = ['blue', 'orange', 'green', 'purple']
    
    for i, param in enumerate(params):
        axs[i].plot(test_data['Time'], test_data[param], color=colors[i])
        axs[i].set_ylabel(param)
        axs[i].set_title(f'{title} - {param}')
        axs[i].grid(True)

    axs[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

def plot_fuel_consumption_bar(data):
    avg_fc = data.groupby('TestID')['FuelConsumption'].mean()
    avg_fc.plot(kind='bar', title='Average Fuel Consumption by Test', ylabel='L/100km')
    plt.xlabel('Test Scenario')
    plt.xticks(rotation=15)
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.show()

def plot_speed_vs_accel(data):
    plt.scatter(data['Speed'], data['Acceleration'], alpha=0.3)
    plt.title('Speed vs. Acceleration (All Tests)')
    plt.xlabel('Speed (km/h)')
    plt.ylabel('Acceleration (m/s²)')
    plt.grid(True)
    plt.show()

# Main Execution
df = load_vehicle_data('sample_data/vehicle_dynamics_data.csv')

if df is not None:
    # Non-interactive part
    for test_id in df['TestID'].unique():
        test_data = df[df['TestID'] == test_id].copy()
        vehicle_key = test_id.split('_')[0].lower()
        test_data = calculate_force(test_data, vehicle_key)
        compute_summary_stats(test_data, test_id)
        braking_events(test_data)
        fe = calculate_fuel_efficiency(test_data)
        print(f"Fuel Efficiency for {test_id}: {fe:.2f} km/l")
        est_dist = estimate_distance_from_speed(test_data)
        print(f"Estimated distance from speed integration: {est_dist:.2f} meters\n")

    # Global plots
    plot_fuel_consumption_bar(df)
    plot_speed_vs_accel(df)

    # User Interaction Loop
    while True:
        user_input = input("View plots for which vehicle? (sedan/suv/sports/exit): ").strip().lower()
        if user_input == 'exit':
            break
        elif user_input in VEHICLE_SPECS:
            selected_test = df[df['TestID'].str.startswith(user_input)]
            if not selected_test.empty:
                selected_test = calculate_force(selected_test.copy(), user_input)
                plot_vehicle_data(selected_test, user_input.title())
            else:
                print("No data found for that vehicle.")
        else:
            print("Invalid input. Try again.")